In [4]:
pip install datasets kaggle PyMuPDF Pillow tf-keras tesseract-ocr

  Using cached tesseract-ocr-0.0.1.tar.gz (33 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Cython-3.0.12-cp312-cp312-win_amd64.whl.metadata (3.6 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/376.0 MB ? eta -:--:--
   ---------------------------------------- 1.6/376.0 MB 7.6 MB/s eta 0:00:50
   ---------------------------------------- 3.7/376.0 MB 8.4 MB/s eta 0:00:45
    --------------------------------------- 6.0/376.0 MB 9.7 MB/s eta 0:00:39
    --------------------------------------- 8.7/376.0 MB 10.5 MB/s eta 0:00:35
   - -------------------------------------- 10.5/376.0 MB 10.2 MB/s eta 0:00:36
   - -------------------------------------- 13.4/376.0 MB 10.6 MB/s eta 0:00:35
   - -------

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
      c:\Users\annap\AppData\Local\Programs\Python\Python312\Lib\site-packages\setuptools\dist.py:493: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              ********************************************************************************
      
      !!
        opt = self.warn_dash_depre

In [2]:
# def setup_kaggle():
#   from google.colab import files
#   uploaded = files.upload()
#   !chmod 600 kaggle.json
#   %mkdir -p /root/.kaggle/
#   %mv kaggle.json /root/.kaggle/

# setup_kaggle()

#RUN THIS COMMAND ON TERMINAL --> TO CONNECT TO KAGGLE  OR RUN HERE WITH OS
import os

os.system(r'icacls "C:\Users\annap\.kaggle\kaggle.json" /inheritance:r /grant:r annap:R')


0

In [3]:
import kaggle
kaggle.api.authenticate()
# import recmetrics

from PIL import Image
import pytesseract
import pandas as pd
import numpy as np
# import fitz
import os
import cv2
import re
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from sklearn.preprocessing import LabelEncoder
# from collections import defaultdict
# from nltk.corpus import wordnet as wn
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn import model_selection, naive_bayes, svm
# from sklearn.metrics import accuracy_score
from transformers import AutoProcessor, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset  #, DatasetDict

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

## From JPG


In [ ]:
dataset_path = 'shaz13/real-world-documents-collections'
kaggle.api.dataset_download_files(dataset_path, path=dataset_path, unzip=True)
print(os.listdir(dataset_path))


Dataset URL: https://www.kaggle.com/datasets/shaz13/real-world-documents-collections
['docs-sm']


In [ ]:
np.random.seed(500)
#invoices
invoice_path = f'{dataset_path}/docs-sm/invoice'
print(os.listdir(invoice_path))


['0000137486.jpg', '0000192502.jpg', '0000226471.jpg', '0000227351.jpg', '0000333206.jpg', '0000542607.jpg', '0001136788.jpg', '0001219743.jpg', '0001219844.jpg', '0001233629.jpg', '0001453135.jpg', '0011519293.jpg', '0011929750.jpg', '0011929754.jpg', '0011930170.jpg', '0011991025.jpg', '0012179618.jpg', '0012193483.jpg', '0012287015.jpg', '0013041942.jpg', '0013043203.jpg', '0013122156.jpg', '0013267193.jpg', '0060026309.jpg', '0060027381.jpg', '0060031784.jpg', '0060032039.jpg', '0060046962.jpg', '0060051594.jpg', '0060075043.jpg', '0060080760.jpg', '0060082098.jpg', '0060083495.jpg', '0060095587.jpg', '0060095589.jpg', '0060116328.jpg', '0060121960.jpg', '0060201834.jpg', '0060207574.jpg', '0060281129.jpg', '0060284223.jpg', '00621985.jpg', '00920638.jpg', '00920717.jpg', '01145176.jpg', '01398469.jpg', '03638969.jpg', '03660266.jpg', '03700650.jpg', '03724966.jpg', '03750991.jpg', '1000031431.jpg', '10399076.jpg', '10399232.jpg', '11235021_11235022.jpg', '11238840.jpg', '200120567

# Utility Functions for LayoutMv3

In [ ]:
def normalize_box(bbox, w, h):
    return [
        int(bbox[0]*(1000/w)),
        int(bbox[1]*(1000/h)),
        int(bbox[2]*(1000/w)),
        int(bbox[3]*(1000/h)),
    ]


def unnormalize_box(bbox, w, h):
    return [
        w * (bbox[0] / 1000),
        hash * (bbox[1] / 1000),
        w * (bbox[2] / 1000),
        h * (bbox[3] / 1000),
    ]


#comparing bounding boxes for debugging
def compare_boxes(b1, b2):
    b1 = np.array([c for c in b1])
    b2 = np.array([c for c in b2])
    equal = np.array_equal(b1, b2)
    return equal


def adjacent(w1, w2):
    if w1['label'] == w2['label'] and abs(w1['id'] - w2['id']) == 1:
      return True
    return False


# def load_model(model_path):
#     model = AutoModelForSequenceClassification.from_pretrained(model_path)
#     return model


# def load_processor():
#     processor = AutoProcessor.from_pretrained(
#         "microsoft/layoutlmv3-base", apply_ocr=False)
#     return processor


# Preprocessing

In [ ]:
'''tesseract runs OCR on images to recognize text'''
def run_tesseract_on_image(image_path):  # -> tsv output path
    # image_path = image_path.replace("\\", "/") 

    base_path = r"C:\Users\annap\Desktop\Year3\stats\final_project" 
    image_path = os.path.join(base_path, image_path)  
    image_path = os.path.normpath(image_path) 
    print("IMAGE PATH", image_path)

    # image_path = r"C:\Users\annap\Desktop\Year3\stats\final_project\"" + image_path
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    #DEBUG
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Unable to read image: {image_path}")   


    tsv_directory = os.path.abspath(os.path.join(os.getcwd(), "tsv"))
    if not os.path.exists(tsv_directory):
        os.makedirs(tsv_directory)

    tsv_output_path = os.path.join(tsv_directory, f"{image_name}.tsv")

    print(f"Running Tesseract on image: {image_path}")
    print(f"Saving TSV to: {tsv_output_path}")
    print(f"Command: tesseract \"{image_path}\" \"{tsv_output_path}\" -l eng tsv")


    tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    error_code = os.system(f'"{tesseract_cmd}" "{image_path}" "{tsv_output_path}" -l eng tsv')

    if not error_code:
      return tsv_output_path              #saves extracted text from image in the tsv file
    else:
      raise ValueError('Error running Tesseract on image/verify image format PNG,JPG,JPEG')



'''reads the file generated above, removes empty/missing entries,
iterates through the cleaned dataset and stores words and bbox coordinates'''
def clean_tesseract_output(tsv_output_path):
    ocr_df = pd.read_csv(tsv_output_path, sep='\t')
    ocr_df = ocr_df.dropna()
    ocr_df = ocr_df.drop(ocr_df[ocr_df.text.str.strip() == ''].index)
    # text_output = ' '.join(ocr_df.text.tolist())

    words = []
    for index, row in ocr_df.iterrows():
      word = {}
      origin_box = [row['left'], row['top'], row['left'] + row['width'], row['top']+row['height']]
      word['word_text'] = row['text']
      word['word_box'] = origin_box
      words.append(word)

    return words          #array of dicts that include bbox coords and words details






# def prepare_batch_for_inference(image_paths):
#     tesseract_outputs = [run_tesseract_on_image(image_path) for image_path in image_paths]    # tesseract_outputs is list of paths
#     clean_outputs = [clean_tesseract_output(tsv_path) for tsv_path in tesseract_outputs]        # clean_outputs is a list of lists

#     word_lists = [[word['word_text'] for word in clean_output] for clean_output in clean_outputs]
#     boxes_lists = [[word['word_box'] for word in clean_output] for clean_output in clean_outputs]

#     inference_batch = {
#       "image_path": image_paths,
#       "bboxes": boxes_lists,
#       "words": word_lists
#     }

#     return inference_batch




In [ ]:
def extract_text_bboxes(image_path, use_tesseract_tsv=True):
    if use_tesseract_tsv:                     #tesseract TSV
      tsv_path = run_tesseract_on_image(image_path)
      words_boxes = clean_tesseract_output(tsv_path)
      words = [item['word_text'] for item in words_boxes]
      bboxes = [item['word_box'] for item in words_boxes]

    # else:                                     #pytesseract
    #   img = cv2.imread(image_path)
    #   gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #   gray = cv2.GaussianBlur(gray, (5,5), 0)
    #   _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #   extracted_text = pytesseract.image_to_string(thresh)

    #   data = pytesseract.image_to_data(thresh, output_type=pytesseract.Output.DICT)   #extracts words, bboxes

    #   words, bboxes = [], []
    #   h, w, _ = img.shape

    #   for i in range(len(data["text"])):
    #     if data["text"][i].strip():
    #       words.append(data["text"][i])
    #       x,y,w,h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
    #       bbox = [
    #           int((x/w)*1000),
    #           int((y/h)*1000),
    #           int(((x + w)/w) * 1000),
    #           int(((y + h)/h) *1000)]      #1000x1000  normalizes bboxes
    #       bboxes.append(bbox)

    return words, bboxes



def clean_text(df):
    # df = df.dropna()
    df = df.explode("tokens")
    df = df[df["tokens"].str.strip() != '']

    # text_output = ' '.join(df.text.tolist())

    words = []
    for index, row in df.iterrows():
        word = {}
        origin_box = row['bboxes']
        word['word_text'] = row['tokens']
        word['word_box'] = origin_box
        words.append(word)

    return words



#prepares for inference
invoice_data_list = []
print(invoice_path)

for filename in os.listdir(invoice_path):
    if filename.lower().endswith(('.jpg', '.jpeg',)):
        print(f"Valid image: {filename}")
        image_path = os.path.join(invoice_path, filename)
        words,bboxes = extract_text_bboxes(image_path, use_tesseract_tsv=True)
        # invoice_data = extract_invoice_data(text)
        # invoice_data["File Name"] = filename
        invoice_data_list.append({
            "file_name": filename,
            "tokens": words,
            "bboxes": bboxes,
        })
    else:
        print(f"Skipping non-image file: {filename}")

df = pd.DataFrame(invoice_data_list)
# print(df)
# print(df.columns)


df = clean_text(df)
print(df)


shaz13/real-world-documents-collections/docs-sm/invoice
Valid image: 0000137486.jpg
IMAGE PATH C:\Users\annap\Desktop\Year3\stats\final_project\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg
Running Tesseract on image: C:\Users\annap\Desktop\Year3\stats\final_project\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg
Saving TSV to: c:\Users\annap\Desktop\Year3\stats\final_project\tsv\0000137486.tsv
Command: tesseract "C:\Users\annap\Desktop\Year3\stats\final_project\shaz13\real-world-documents-collections\docs-sm\invoice\0000137486.jpg" "c:\Users\annap\Desktop\Year3\stats\final_project\tsv\0000137486.tsv" -l eng tsv


ValueError: Error running Tesseract on image/verify image format PNG,JPG,JPEG

LayoutMv3 needs textual and spatial information. So, we need the bounding boxes of where the text is positioned within the picture.

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

#convert into huggnig face dataset
hf_dataset = Dataset.from_pandas(df)
print(hf_dataset[0])   #check it worked


#passes words/bboxes into the layoutmv3 processer
def preprocess(example, img_path):
  img = Image.open(image_path.convert("RBG"))
  encoding = processor(
      images=img,
      text=example["tokens"],
      boxes=example["bboxes"],
      truncation=True,
      padding="max_length"
  )
  return encoding


dataset = hf_dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]


# Invoice Extraction

In [ ]:
# we will need to recreate this for all categories
def extract_invoice_data(text):
    data = {}

    data["Invoice Number"] = re.search(r'Invoice\s*No[:#]?\s*([\w-]+)', text, re.IGNORECASE)
    data["Invoice Date"] = re.search(r'Invoice\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Due Date"] = re.search(r'Due\s*Date[:#]?\s*([\d{2}/\d{2}/\d{4}]+)', text, re.IGNORECASE)
    data["Issuer Name"] = re.search(r'From[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Recipient Name"] = re.search(r'To[:#]?\s*([A-Za-z\s]+)', text, re.IGNORECASE)
    data["Total Amount"] = re.search(r'Total\s*Amount[:#]?\s*([\d,]+\.?\d{2})', text, re.IGNORECASE)

    for key, value in data.items():
        data[key] = value.group(1) if value else "Not Found"

    return data


# LayoutMv3

In [ ]:
#pretrained layoutmv3 model
model = AutoModelForSequenceClassification("microsoft/layoutlmv3-base", num_labels=4)

train_args = TrainingArguments(output_dir="./layoutlmv3_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs"
    )


trainer= Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
    )


trainer.train()

